---
title: Ion Scattering by Solar Wind Discontinuities
---

Check the properties related to ion scattering by solar wind discontinuities.


We need to use GLMakie because the volume is not supported by CairoMakie yet.

In [7]:
using Revise
using DrWatson
@quickactivate
include(srcdir("main.jl"))
includet(srcdir("scattering.jl"))
using Dates

In [2]:
v0 = :v0 => log2 => "Log2 v0";

In [9]:
name = :wind
df = load(name);
v_norm!(df);
layer = data(df) * mapping(layout = :accuracy);

In [4]:
foreach([:log10, :log2]) do f
    @eval import Base: $f
    @eval $f(x::Unitful.Quantity) = $f(ustrip(x))
end

In [58]:
datalimits = (2, 14)
axis = (;yscale = log10)
v = histogram(;normalization=:pdf, bins=20, datalimits)
dfplot(layer * v, v0; axis)

In [11]:
using FHist
using LinearAlgebra

function get_data(df; x=:v0, y=:ω_in, z=:θ)
    x = df[!, x] .|> ustrip .|> log2
    y = df[!, y]
    z = df[!, z] .|> abs
    return x, y, z
end

function get_hist3d(df)
    return Hist3D(
        get_data(df);
        binedges=(1.5:14.5, 20:10:180, 0:10:90),
    ) |> normalize
end

function plot_hist3d(df)
    h = get_hist3d(df)
    fig = Figure()
    ax = Axis3(fig[1, 1]; width=400, xlabel="log2(v0)", ylabel="ω_in", zlabel="θ")
    plot!(h)
    fig
end

using GLMakie
GLMakie.activate!()
plot_hist3d(filter_low_accuracy(df))

┌ Warning: Encountered an `AbstractVector` with value [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5] on side x in `convert_arguments` for the `ImageLike` trait.
│         Using an `AbstractVector` to specify one dimension of an `ImageLike` is deprecated because `ImageLike` sides always need exactly two values, start and stop.
│         Use interval notation `start .. stop` or a two-element tuple `(start, stop)` instead.
└ @ Makie /Users/zijin/.julia/packages/Makie/6c4lt/src/conversions.jl:416
┌ Warning: Encountered an `AbstractVector` with value [20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 110.0, 120.0, 130.0, 140.0, 150.0, 160.0, 170.0, 180.0] on side y in `convert_arguments` for the `ImageLike` trait.
│         Using an `AbstractVector` to specify one dimension of an `ImageLike` is deprecated because `ImageLike` sides always need exactly two values, start and stop.
│         Use interval notation `start .. stop` or a two-element tuple `(start, 

In [12]:
using HDF5

filename = datadir("$(name)_hist3d.h5")
h = get_hist3d(filter_low_accuracy(df))

function save(filename, h::Hist3D)
    # delete file if it exists
    isfile(filename) && rm(filename)
    h5writehist(filename, "hist", h)
end

save(filename, h)

In [15]:
*(size(bincounts(h))...) |> println
bincenters(h)

1872


([2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0], [25.0, 35.0, 45.0, 55.0, 65.0, 75.0, 85.0, 95.0, 105.0, 115.0, 125.0, 135.0, 145.0, 155.0, 165.0, 175.0], [5.0, 15.0, 25.0, 35.0, 45.0, 55.0, 65.0, 75.0, 85.0])

In [52]:
xyz = (v0, ω_in, θ)
datalimits = ((1,14), (0,180), (0,180))
axis = (type = Axis3, width = 400)
v =  histogram(;datalimits, normalization=:pdf)
plt = data(df) * mapping(xyz...; layout = :accuracy) * v
fg = draw(plt; axis)
colorbar!(fg)
easy_save("scatter/$(name)_hist3d"; formats=[:png], dir=figure_dir)

┌ Warning: Encountered an `AbstractVector` with value [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5] on side x in `convert_arguments` for the `ImageLike` trait.
│         Using an `AbstractVector` to specify one dimension of an `ImageLike` is deprecated because `ImageLike` sides always need exactly two values, start and stop.
│         Use interval notation `start .. stop` or a two-element tuple `(start, stop)` instead.
└ @ Makie /Users/zijin/.julia/packages/Makie/6c4lt/src/conversions.jl:416
┌ Warning: Encountered an `AbstractVector` with value [5.0, 15.0, 25.0, 35.0, 45.0, 55.0, 65.0, 75.0, 85.0, 95.0, 105.0, 115.0, 125.0, 135.0, 145.0, 155.0, 165.0, 175.0, 185.0] on side y in `convert_arguments` for the `ImageLike` trait.
│         Using an `AbstractVector` to specify one dimension of an `ImageLike` is deprecated because `ImageLike` sides always need exactly two values, start and stop.
│         Use interval notation `start .. stop` or a two-element tupl